In [1]:
# loading the code
include("../src/DifferenceZeroTest.jl")


Welcome to Nemo version 0.18.2

Nemo comes with absolutely no warranty whatsoever



In this notebook, we will consider the following relation between the gamma function, Barnes G-function, and the log-gamma integral:

$\Lambda(n) := \int\limits_0^n \ln\Gamma(x) \operatorname{dx} = \frac{n(n - 1)}{2} + \frac{n}{2} \ln 2\pi + n \ln \Gamma(n) - \ln G(n + 1)$

We will prove it by using the following difference equations satisfied by $\ln \Gamma(x)$, $\ln G(x + 1)$, and $\Lambda(n)$:
* $\ln \Gamma(x + 1) = \ln x + \ln \Gamma(x)$;
* $\ln G(x + 2) = \ln \Gamma(x + 1) + \ln G(x + 1)$;
* $\Lambda(x + 1) = \Lambda(x) + x \ln x - x + \Lambda(1)$, where $\Lambda(1) = \frac{1}{2}\ln 2\pi$
and translating them to the equations for the asymptotic expansions at infinity.

First we adjoing the asymptotic series for the log-gamma function in the same way as in [this notebook](https://github.com/pogudingleb/DifferenceZeroTest/blob/main/examples/LegendreDuplication.ipynb):

In [2]:
# define the shift
X = IdentityPS{fmpq}(Nemo.QQ)
S = X / (1 + X)

# adjoining z
polyring, (x0, x1, x2) = PolynomialRing(Nemo.QQ, ["x0", "x1", "x2"])
z = AnnihilatorBasedPS{fmpq}(x1 * (1 + x0) - x0, S, [0, 1, 0, 0])
D = construct_domain(z)
z = gen(D)

# adjoining log(1 + z)
D = adjoin_transcendental(D, LogPS{fmpq}(Nemo.QQ))
logz = gen(D)

# adjoining the Stirling series S(z)
polyring2, (x0, x1) = PolynomialRing(D, ["x0", "x1"])
annihilator = z * (x1 - x0) - z + (1 + z) * logz - 1//2 * z * logz
stirling = AnnihilatorBasedPS{fmpq}(annihilator, S, [0, 1//12, 0, -1//360, 0, 1//1260, 0])
D = adjoin_sigma_algebraic(D, stirling)
stirling = gen(D);

Using the following asymptotic expansion 

$\ln G(x + 1) \sim \frac{x^2}{2}\ln x - \frac{3x^2}{4} + \frac{x}{2}\ln 2\pi - \frac{1}{12}\ln x + C + B\left(\frac{1}{x}\right)$

and combining it with the equation $\ln G(x + 2) = \ln \Gamma(x + 1) + \ln G(x + 1)$, we can adjoin the series $B(z)$ to our domain:

In [3]:
polyring2, (x0, x1) = PolynomialRing(D, ["x0", "x1"])
annihilator = z^2 * (x1 - x0 - stirling) + 1//2 * (1 + z)^2 * logz - 1//12 * z^2 * logz - 1//2 * z - 3//4 * z^2
barnes = AnnihilatorBasedPS{fmpq}(annihilator, S, [0, 0, -1//(30 * 8), 0, 1//(42 * 24), 0])
D = adjoin_sigma_algebraic(D, barnes)
barnes = gen(D);

Finally, we use the asymptotic expansion for $\Lambda(x)$

$\Lambda(x) \sim \frac{x^2}{4} - \frac{x}{2} + \frac{x^2}{2}\ln x - \frac{1}{2}x\ln x + \frac{1}{12}\ln x + C + L\left(\frac{1}{x} \right)$

and combine with $\Lambda(x + 1) = \Lambda(x) + x \ln x - x + \frac12 \ln 2\pi$ to adjoin the series $L(z)$ to our domain:

In [4]:
polyring2, (x0, x1) = PolynomialRing(D, ["x0", "x1"])
annihilator = z^2 * (x1 - x0) - 1//2 * z - 1//4 * z^2 + logz * (1//2 + 1//2 * z + 1//12 * z^2)
loggammaint = AnnihilatorBasedPS{fmpq}(annihilator, S, [0, 0, 1//(6 * 120), 0, -1//5040])
D = adjoin_sigma_algebraic(D, loggammaint)
loggammaint = gen(D);

In terms of the series $S(z), B(z), L(z)$, the identity of interest can be written as

$L(z) = \frac{1}{z}S(z) - B(z) - \frac{1}{12}z$

and verified using our algorithm:

In [5]:
identity = z * loggammaint + z * barnes - stirling + 1//12 * z
iszero(identity)

true